# NOC processing
#### This is the fourth and final step of the enrichment calculation
Calculate NOC proportion and append to the enrichment table  

Key output files
- `yyyy-mm-dd_enrichment_table_NOC_prop.csv`: enrichment values and NOC proportion  
- `yyyy-mm-dd_volcano_table.csv`: enrichment values and p-values  
  
both files are in the directory `output/enrichment_and_volcano_tables`

In [1]:
import sys
from pathlib import Path
import pandas as pd

script_path = Path.cwd().parent.parent.parent.parent / "script"
data_path = Path.cwd().parent.parent.parent.parent / "data"
sys.path.append(str(script_path))

from pyseus import basic_processing as ip
from pyseus import contrast_tools as ct

### Load imputed NOC/Bulk table
The correct datestamp is required to find the input files

In [2]:
%store -r fig5_timestamp FIG5_USE_FROZEN
if FIG5_USE_FROZEN:
    raise Exception("USE_FROZEN is true, you probably want to skip enrichment and proceed from 3.aligned_umap")
timestamp = fig5_timestamp
print(f"Timestamp: {timestamp}")

outprefix = f"{timestamp}_QC_filter_impute"

outdir = Path.cwd() / "output"
preprocessing_out = outdir / f"preprocessing"

NOC_path = preprocessing_out / f"{outprefix}_filtered_table.csv"

try:
    noc_table = pd.read_csv(NOC_path, header=[0], index_col=0)
except FileNotFoundError:
    print(f"File {NOC_path} not found.\nPlease rerun the previous steps or specify the correct timestamp, current value is {timestamp}")
except pd.errors.ParserError:
    print(f"There was an error parsing the CSV file at {NOC_path}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Timestamp: 2024-10-27


### Calculation of proportion

In [3]:
# select the noc columns
NOC_kw = ["Cytosol", "Nuclear", "Organelle"]
noc_cols = [
    col for col in list(noc_table)
    if (any(kw in col for kw in NOC_kw)) and "Proteome" not in col
]
print(noc_cols)

['Infected_Cytosol_1', 'Infected_Cytosol_2', 'Infected_Cytosol_3', 'Infected_Nuclear_1', 'Infected_Nuclear_2', 'Infected_Nuclear_3', 'Infected_Organelle_1', 'Infected_Organelle_2', 'Infected_Organelle_3']


In [4]:
# select the noc_cols + Protein IDs for merge
noc_table = noc_table[["Protein IDs"] + noc_cols].copy()

In [5]:
# group the tables and take the median

# start RawTables class from basic processing to group the samples
noc_process = ip.RawTables(
    proteingroup=None, file_designated=True,
    info_cols=['Protein IDs'], sample_cols=noc_cols
)

# designate the pre-computed noc filtered table on this class
noc_process.filtered_table = noc_table

# group replicates
noc_process.group_replicates(reg_exp=r'(.*)_\d+$')

# calculate the median
noc_medians = ip.median_replicates(noc_process.grouped_table)
noc_medians.head()

Intensity values have not been transformed yet from filtered table,
we recommend using transform_intensities() method before grouping replicates.

Using filtered_table to group replicates.


,Infected_Organelle,Infected_Cytosol,Infected_Nuclear,Protein IDs
2,100650000.0,116740000.0,267320000.0,A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124
3,170150000.0,171030000.0,302530000.0,Q9Y5S9;A0A023T787;A0A0J9YW13
5,45439000.0,2313300.0,14070000.0,A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...
6,1104400.0,2772200.0,0.0,A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...
7,55897000.0,3596800.0,30991000.0,Q99805;A0A024QYR8;B3KSG9


In [6]:
# we take the sum of each column 
noc_medians['noc_sum_inf'] = noc_medians['Infected_Cytosol'] + noc_medians['Infected_Organelle'] + noc_medians['Infected_Nuclear']

# remove all proteingroups where both inf and uninf noc_sum is 0 -> there is no information on these
noc_medians = noc_medians[(noc_medians['noc_sum_inf'] > 0)].reset_index(drop=True)

# create a new dataframe with just the proportion: intensity divided by the sum
noc_proportion = noc_medians[['Protein IDs']].copy()
noc_proportion['NOC_cytosol_Infected'] = noc_medians['Infected_Cytosol'] / noc_medians['noc_sum_inf']
noc_proportion['NOC_organelle_Infected'] = noc_medians['Infected_Organelle'] / noc_medians['noc_sum_inf']
noc_proportion['NOC_nuclear_Infected'] = noc_medians['Infected_Nuclear'] / noc_medians['noc_sum_inf']


### Merge NOC proportion with IP enrichment

In [7]:
# read the enrichment table
enrich_out_dir = outdir / "enrichment_and_volcano_tables"
enrichment_csv_path = enrich_out_dir / f'{timestamp}_enrichment_table.csv'
enrichments = pd.read_csv(enrichment_csv_path, header=[0, 1], index_col=0)
meta_cols = enrichments["metadata"].columns
enrichments = enrichments.droplevel(0, axis=1)

In [8]:
# finally, merge with the enrichment table
enrichment_nocs = enrichments.merge(noc_proportion, on='Protein IDs', how='inner')

# standard formatting
enrichment_nocs = ct.standard_pyseus_headers(enrichment_nocs, meta_cols=meta_cols)

In [9]:
# save the enrichment table with NOC proportions
enrichment_NOC_csv_path = enrich_out_dir / f'{timestamp}_inf_enrichment_table_NOC_prop.csv'
enrichment_nocs.to_csv(enrichment_NOC_csv_path)

In [10]:
# enrichment table with NOC proportions
enrichment_nocs

metadata  \
                                            Protein IDs   
0         A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124   
1                          Q9Y5S9;A0A023T787;A0A0J9YW13   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...   
4                              Q99805;A0A024QYR8;B3KSG9   
...                                                 ...   
8371   X5D2T3;X1WI28;P27635;B8A6G2;X5D2W5;A6QRI9;Q96L21   
8372                                             X5D7P8   
8373  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...   
8374                                             X5DQZ7   
8375                                             X6RHX1   

                                                                          \
                                   Majority protein IDs       Gene names   
0                       A0A023T6R1;Q96A72;F5H6P7;F5H6N1  FLJ10292;MAGOHB   
1                                     Q9Y5S9;A0A023T787       RBM8A;RBM8   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6ZTK5           TM9SF4   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...             PTEN   
4                              Q99805;A0A024QYR8;B3KSG9           TM9SF2   
...                                                 ...              ...   
8371                               X5D2T3;X1WI28;P27635            RPL10   
8372                                             X5D7P8            STK39   
8373  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...            CRMP1   
8374                                             X5DQZ7             GPX1   
8375                                             X6RHX1            SCYL3   

                   sample                                                     \
     17-MAP1LC3B_Infected 12-WT_Infected 14-RAB7A_Infected 14-RAB14_Infected   
0               -0.914112      -0.147902          0.144060          1.801515   
1                1.988100       1.549150         -0.607750         -0.487250   
2                0.814900      -1.216500          2.556450          2.810350   
3                0.173554      -0.339343         -0.124471         -0.336023   
4                1.631900      -3.085650          2.033150          2.474550   
...                   ...            ...               ...               ...   
8371            -0.049800       0.676750         -0.987550         -0.719850   
8372            -0.262306       0.883674         -0.980282          0.115286   
8373             0.337399      -0.253840          0.450581          0.005724   
8374            -0.732131      -0.436745          1.369150          4.366004   
8375             0.377157      -0.352498          0.259682          0.552513   

                                                          ...  \
     14-WT_Infected 14-COPE_Infected 17-SLC30A2_Infected  ...   
0          1.451371        -0.402976            0.689794  ...   
1          0.253300        -1.376200           -1.118800  ...   
2          1.217900         2.939100           -3.445300  ...   
3          0.990840         1.818102           -0.354000  ...   
4          0.645950         1.628800           -1.637100  ...   
...             ...              ...                 ...  ...   
8371      -0.881050        -0.467100            0.746000  ...   
8372      -0.052230        -0.131317           -0.298151  ...   
8373      -1.006133        -0.496490           -0.205277  ...   
8374       2.537904        -0.970853           -1.772940  ...   
8375      -0.517810         6.480915           -0.254038  ...   

                                                                           \
     10-EXOC2_Infected 10-RTN4_Infected 11-SEC31A_Infected 10-WT_Infected   
0             1.807292        -1.510986           0.007503       1.137843   
1             1.100000        -1.694300           0.352900       0.557200   
2            -0.819500        -1.205400           1.233000       0.337500   
3            -1.24